# **Template OP on salish**

In [30]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, AdvectionRK4, AdvectionRK4_3D

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels import DeleteParticle, Buoyancy
from OP_functions import *

In [31]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

## Definitions

In [32]:
start = datetime(2019, 12, 1) #Start date
# Set Time length [days] and timestep [seconds]
length = 30
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = 2 # number of deploying locations
n = 150 # 1000   # number of particles per location
dmin = [59,64] #minimum depth
dd = 2 #max depth difference from dmin
r = 1000 #radius deploy cloud [m]
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd,r)

In [33]:
from parcels import Variable

class PEParticle(JITParticle):         # Define a new particle class
    ro = Variable('ro', initial = 1370)             # desnsity PE
    size = Variable('size', initial = 5e-5)

In [34]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
# Execute run 
clon, clat = [-123.407687,-123.347869], [48.406743,48.393512]  # choose horizontal centre of the particle cloud
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [35]:
#Set start date time and the name of the output file
name = 'VictoriaWinter' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/analysis-jose/notebooks/results/VictoriaWinter20191201_1n_20191231_1n.nc


## Simulation

In [36]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

In [37]:
# #Load Salish output as fields
field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, JITParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

#p_flt = pset.Kernel(Buoyancy) #uncomment if you want to use additional kernels, also change the value of pset.Kernel to "AdvectionRK4_3D + p_flt"
##pset.computeTimeChunk(allow_time_extrapolation=1)
pset.execute(AdvectionRK4_3D, 
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayJITParticleAdvectionRK4_3D ==> /tmp/parcels-2894/libd477aeada3ff6edf4c5a81edf938ec2b_0.so
INFO: Temporary output files are stored in /home/jvalenti/MOAD/analysis-jose/notebooks/results/out-SBTBSOPN.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/analysis-jose/notebooks/results/out-SBTBSOPN" to convert these to a NetCDF file during the run.
  2% (54000.0 of 2592000.0) |            | Elapsed Time: 0:00:10 ETA:   0:18:20/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)
 22% (583200.0 of 2592000.0) |##         | Elapsed Time: 0:04:21 ETA:   0:15:27

Correct cell not found for (-124.845087, 48.576768) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 435 0
            new particle indices: (yi, xi) 434 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.2998703077007328e+01 -4.7519815204073397e-02
Particle 788 lost !! [-124.84502540424855, 48.576694049822265, 6.084767383637086, 587250.0]


 22% (588600.0 of 2592000.0) |##         | Elapsed Time: 0:04:25 ETA:   0:50:46

Correct cell not found for (-124.838381, 48.568783) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 432 0
            new particle indices: (yi, xi) 432 43
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3980994672715909e+01 9.7342680144350968e-01
Particle 773 lost !! [-124.83833883060036, 48.56873276985747, 22.417402575049252, 594180.0]


 22% (594000.0 of 2592000.0) |##         | Elapsed Time: 0:04:27 ETA:   0:14:13

...

 97% (2527200.0 of 2592000.0) |######### | Elapsed Time: 0:18:59 ETA:   0:00:28

Correct cell not found for (-124.761547, 48.477239) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 409 0
            new particle indices: (yi, xi) 409 43
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3985035150186548e+01 9.5561158013898895e-01
Particle 681 lost !! [-124.76139138077518, 48.477054334085814, 61.08077133773523, 2531700.0]


 97% (2530800.0 of 2592000.0) |######### | Elapsed Time: 0:19:00 ETA:   0:00:23

Correct cell not found for (-124.738155, 48.449345) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 402 0
            new particle indices: (yi, xi) 402 43
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.4018324421231789e+01 9.9254965942920825e-01
Particle 824 lost !! [-124.73813496821836, 48.44932106014439, 27.993037023368146, 2534850.0]


100% (2592000.0 of 2592000.0) |##########| Elapsed Time: 0:19:23 Time:  0:19:23
/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)
